# Geospatial Python
## Raster Calculations in Python
Setup: https://carpentries-incubator.github.io/geospatial-python/index.html

Instruction: https://carpentries-incubator.github.io/geospatial-python/09-raster-calculations.html

Objectives:
* Carry out operations with two rasters using Python’s built-in math operators.
* Reclassify a continuous raster to a categorical raster.

We will use the results of the satellite image search 'search.json', which was generated in 'Raster Data Access.ipynb'.

Before executing the code cells, be sure to replace the "_____" as appropriate

In [ ]:
# Import required libraries
import pystac # to load rasters from the search result
import rioxarray # to open and download remote raster data

import geojson # to parse spatial data format
import folium # to create an interactive map
from folium.plugins import Draw # to allow drawing
from localtileserver import TileClient, get_folium_tile_layer # to visualize the geotiff 

import numpy as np # to work with numbered lists
import xarray # to preserve spatial metadata when working with numbered lists

import earthpy.plot as ep # for drawing a legend
import matplotlib.pyplot as plt # for plotting
from matplotlib.colors import ListedColormap # to color our classified data

In [ ]:
# Load the results of our initial imagery search using pystac
items = pystac.ItemCollection.from_file("search.json")

In [ ]:
# Select the second item, and extract the URIs of the red and nir08 bands (“red” and “nir08”, respectively):
red_href = items[1].assets["red"].href
nir_href = items[1].assets["nir08"].href

In [ ]:
# load the rasters with open_rasterio using the argument masked=True.
red = rioxarray.open_rasterio(red_href, masked=True)
nir = rioxarray.open_rasterio(nir_href, masked=True)

In [ ]:
# Let's use an interctive map to create an area of interest (AOI) for use in subsetting our data. 

# Create tiles client for our geotiff
nir_tiles = TileClient(nir_href) # create tiles client
nir_layer = get_folium_tile_layer(nir_tiles, name='nir') # create elevation tile layer

# Create a map object
m = folium.Map( zoom_start=12)

# Add our tiles
nir_layer.add_to(m)

# Add drawing tools
draw = Draw(export=True)
draw.add_to(m)

# Add drawing controls
folium.LayerControl().add_to(m)

m

# Use the draw rectangle tool to create a shape that overlaps a portion of the boundary

In [ ]:
# Copy the geojson from the drawn polygon (click the shape, and copy the text starting from '{"type":"Polygon"', up until the last '}').
geom ='''{"type":"Polygon","coordinates":[[[-106.020813,40.726446],[-106.020813,41.261291],[-105.227051,41.261291],[-105.227051,40.726446],[-106.020813,40.726446]]]}'''

geojson.loads(geom)
cropping_geometries = [geojson.loads(geom)] # converts to list

# https://corteva.github.io/rioxarray/html/rioxarray.html#rioxarray-rio-accessors
red_clip = red.rio.clip(geometries=cropping_geometries, crs=4326)
nir_clip = nir.rio.clip(geometries=cropping_geometries, crs=4326)

In [ ]:
# plot the two rasters Using robust=True
red_clip.plot(robust=True)

In [ ]:
# plot the two rasters Using robust=True
nir_clip.plot(robust=True)

## Raster Math

In [ ]:
#check the shapes of the two rasters in the following way
print(red_clip.shape, nir_clip.shape)

In [ ]:
# As their width and height do not match, use reproject_match to both reproject and clip a raster to the CRS and extent of another raster.
red_clip_matched = red_clip.rio.reproject_match(nir_clip,nodata=np.nan ) # Set NaN as NoData
print(red_clip_matched.shape)

In [ ]:
# Compute the NDVI to create a new raster 
ndvi = (nir_clip - red_clip_matched)/ (nir_clip + red_clip_matched)
print(ndvi)

In [ ]:
# Plot the output NDVI
ndvi.plot()

In [ ]:
# Plot a histogram to see the spread of values accross 50 bins
ndvi.plot.hist(bins=50)

In [ ]:
# Discretize the color plot by specifying the intervals
class_bins = (-1, 0., 0.2, 0.7, 1)
ndvi.plot(levels=class_bins)

In [ ]:
# Missing values can be interpolated from the values of neighbouring grid cells using the .interpolate_na method. 
# ndvi_nonan = ndvi.interpolate_na(dim="x")

# # save the output
# ndvi_nonan.rio.to_raster("NDVI.tif")

## Classifying Continuous Rasters in Python

Reduce the complexity of the map by classifying it. 

Classification involves assigning each pixel in the raster to a class based on its value. 

In Python, we can accomplish this using the *numpy.digitize* function

Note: by default, each class includes the left but not the right bound. This is not an issue here, since the computed range of NDVI values are fully contained in the open interval (-1; 1)

In [ ]:
# Define the bins for pixel values
class_bins = (-1, 0., 0.2, 0.7, 1)

# The numpy.digitize function returns an unlabeled array, in this case, a
# classified array without any metadata. That doesn't work--we need the
# coordinates and other spatial metadata. We can get around this by using
# "xarray.apply_ufunc", which can run the function across the data array while preserving metadata.
ndvi_classified = xarray.apply_ufunc(
    np.digitize,
    ndvi,#ndvi_nonan,
    class_bins,
    dataset_fill_value=np.nan
)

In [ ]:
# Visualize the classified NDVI, customizing the plot with proper title and legend

# Define a color map for the map legend
ndvi_colors = ["blue", "gray", "green", "darkgreen"]
ndvi_cmap = ListedColormap(ndvi_colors)

# Define class names for the legend
category_names = [
    "Water",
    "No Vegetation",
    "Sparse Vegetation",
    "Dense Vegetation"
]

# We need to know in what order the legend items should be arranged
category_indices = list(range(len(category_names)))

# Make the plot
im = ndvi_classified.plot(cmap=ndvi_cmap, add_colorbar=False)
plt.title("Classified NDVI")

# earthpy helps us by drawing a legend given an existing image plot and legend items, plus indices
ep.draw_legend(im_ax=im, classes=category_indices, titles=category_names)

# Save the figure
plt.savefig("NDVI_classified.png", bbox_inches="tight", dpi=300)

In [ ]:
# Export the classified NDVI raster object to a GeoTiff
ndvi_classified.rio.to_raster("NDVI_classified.tif", dtype="int32")

In [ ]:
ndvi_classified.plot.hist()